In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math
import os

In [ ]:
sample_rate = 30000 # true sample rate, usually around 30074-30075, get it from 01_ephys_sychronizaiton_XDAQ.ipynb
lag_time = 0 # the time that ephys starts later than video. negative means ephys starts frst. get it from 01_ephys_sychronizaiton_XDAQ.ipynb
spike_dir = "ephys" # 

In [ ]:
spike_clusters = pd.DataFrame(data=np.load(f"{spike_dir}\spike_clusters.npy"), columns = {"clusters_id"})
spike_times = pd.DataFrame(data=np.load(f"{spike_dir}\spike_times.npy"), columns = {"times"})
cluster_info = pd.read_csv(f"{spike_dir}\cluster_info.tsv", sep='\t', header=0)
spike = spike_clusters.join(spike_times)
print(spike)
print(cluster_info)

In [ ]:
# get group classification -good, multi or noise
# only save good
good_group = cluster_info.loc[cluster_info['group']=='good'][['cluster_id','ch']]
good_group.reset_index(drop=True,inplace=True)

print(good_group)

In [ ]:
# save each group in a .npy file in a new folder

# Create a new folder
folder_name = "spike-time_good"
os.makedirs(folder_name, exist_ok=True)

# Save the array to an .npy file inside the folder
for index,row in good_group.iterrows():
    spike_name = f"cluster{int(row['cluster_id'])}_ch{int(row['ch'])}"
    spike_save = (spike[spike['clusters_id']==row['cluster_id']]['times'] + lag_time) / sample_rate
    file_path = os.path.join(folder_name, spike_name)
    print(len(spike_save))
    np.save(file_path, spike_save)

In [ ]:
# # save to excel (1 group per sheet)
# with pd.ExcelWriter("spike_good.xlsx", mode='w', engine='openpyxl') as writer:
#     for index in good_group:
#         each_group = spike[spike['clusters_id']==index]['times'].to_frame()/sample_rate
#         each_group.rename(columns={'times': index}, inplace=True)
#         each_group.to_excel(writer,sheet_name='group%s'%str(index),index=False)

In [ ]:
# # scatter
# for index in good_group:
#         each_group = spike[spike['clusters_id']==index]['times'].to_frame()/sample_rate
#         each_group.rename(columns={'times': index}, inplace=True)
#         plt.scatter(each_group,np.full(len(each_group),index),s=1)

In [ ]:
# max_time = math.ceil(max(spike['times'])/sample_rate)
# bin = 0.1 # set time window as 0.1s for spike count
# count = math.ceil(max_time/bin)
# spike_rate = pd.DataFrame()

# for index in good_group:
#     each_group = spike[spike['clusters_id']==index]['times'].to_frame()/sample_rate
#     group_rate = np.array([])
#     time = 0
#     for time_index in range(0,count):
#         spikecount = each_group[(each_group['times']>=time) & (each_group['times']<time+bin)].count()/bin
#         group_rate = np.append(group_rate,spikecount)
#         time = time + bin
#     group_rate = pd.DataFrame(data=group_rate,columns={index})
#     if spike_rate.empty:
#         spike_rate = group_rate
#     else:
#         spike_rate = spike_rate.join(group_rate)

# spike_rate.set_index(np.arange(0,max_time,bin),inplace=True)
# spike_rate.to_excel("spike_rate.xlsx")
# spike_rate
            
            